This file can be used to price Azure OpenAI and Azure AI Search for RAG-based use-cases.

Reference to the Azure Services:
1. Azure OpenAI : https://learn.microsoft.com/en-us/azure/ai-services/openai/overview
2. Azure AI Search : https://learn.microsoft.com/en-us/azure/search/search-what-is-azure-search
3. RAG Model : https://learn.microsoft.com/en-us/azure/search/retrieval-augmented-generation-overview

The prices below are based on Australia East Region.
Azure Pricing Calculator was used : https://azure.microsoft.com/en-au/pricing/calculator
The prices are in AUD and are subject to change.

In [228]:
# AI Search Storage Tier Costs per month
aiSearchStorageTierBasicCost = 148.76
aiSearchStorageTierStandardS1Cost = 496.62
aiSearchStorageTierStandardS2Cost = 1984.25
aiSearchStorageTierAdvancedCost = 3972.97
aiSearchStorageTierStorageOptimizedL1Cost = 4293.99
aiSearchStorageTierStorageOptimizedL2Cost = 8586.85

# AI Search Semantic Ranking Costs per 1000 queries and first 1000 queries are free
aiSearchSemanticRankerCost = 1.53


In [229]:
# Azure OpenAI Cost based on GPT-3.5-Tier-0124-16K per 1000 tokens
azureOpenAIInputTokenCost = 0.0008
azureOpenAIOutputTokenCost = 0.0023

In this section we will start to model a contact center agent based RAG use-cases. The following are the parameters we will use

1. Number of agents e.g. 100
2. How much time it takes to address a customer query e.g. 5 mins
3. How many words used in an input query | Assumption 1 word = 0.75 tokens | https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
4. How many words in a query response | Assumption 1 word = 0.75 tokens | https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
5. working days in a month e.g. 20

In [230]:
numberOfUseCases = 16
numberOfAgents = 300
azureDiscount = 0.325
storageTierCost = aiSearchStorageTierStorageOptimizedL2Cost


timeToAnswerCustomerQueryMins = 5
wordsInputQuery = 2500
wordsOutputQuery = 500
workingHoursInADay = 8
workingDaysInMonth = 20
tokensInAWord=1.34

In [231]:
#perHourCost
costOfInputTokensPerHour = numberOfAgents * (60.0 / timeToAnswerCustomerQueryMins) * ((wordsInputQuery * tokensInAWord)/1000.0) * azureOpenAIInputTokenCost
costOfOutputTokensPerHour = numberOfAgents * (60.0 / timeToAnswerCustomerQueryMins) * ((wordsOutputQuery * tokensInAWord)/1000.0) * azureOpenAIOutputTokenCost
numberOfSearchQueriesPerHour = numberOfAgents * (60.0 / timeToAnswerCustomerQueryMins)

#perMonthCost
costOfInputTokensPerMonth = costOfInputTokensPerHour * workingHoursInADay * workingDaysInMonth
costOfOutputTokensPerMonth = costOfOutputTokensPerHour * workingHoursInADay * workingDaysInMonth
costNumberOfSearchQueriesPerMonth = (((numberOfSearchQueriesPerHour * workingHoursInADay * workingDaysInMonth)-1000)/1000) * aiSearchSemanticRankerCost
totalCostPerMonth = storageTierCost + costOfInputTokensPerMonth + costOfOutputTokensPerMonth + costNumberOfSearchQueriesPerMonth

In [232]:
print("Azure AI Search : Storage Tier Cost Per Month", storageTierCost)
print("Azure AI Search : Semantic Ranker Cost Per Month",costNumberOfSearchQueriesPerMonth)
print("Azure OpenAI : Input Token Cost Per Month",costOfInputTokensPerMonth)
print("Azure OpenAI : Output Token Cost Per Month",costOfOutputTokensPerMonth)
print("Total Azure AI Search Cost",storageTierCost + costNumberOfSearchQueriesPerMonth)
print("Total Azure OpenAI Cost",costOfInputTokensPerMonth + costOfOutputTokensPerMonth)
print("Total Cost Per Month for One Use-Case",totalCostPerMonth)
print("\n---------------Multiple Similar Use-Cases--------------------")
print("Total cost for Azure AI Search for", numberOfUseCases, "UseCases", (storageTierCost + costNumberOfSearchQueriesPerMonth) * numberOfUseCases)
print("Total cost for Azure OpenAI for", numberOfUseCases, "UseCases", (costOfInputTokensPerMonth + costOfOutputTokensPerMonth) * numberOfUseCases)
print("Total cost for", numberOfUseCases, "UseCases",numberOfUseCases*totalCostPerMonth)
print("\n---------------After Discount--------------------")
print("Total cost for Azure AI Search for", numberOfUseCases, "UseCases", ((storageTierCost + costNumberOfSearchQueriesPerMonth) * numberOfUseCases) * (1-azureDiscount))
print("Total cost for Azure OpenAI for", numberOfUseCases, "UseCases", ((costOfInputTokensPerMonth + costOfOutputTokensPerMonth) * numberOfUseCases) * (1-azureDiscount))
print("Total cost for", numberOfUseCases, "UseCases",(numberOfUseCases*totalCostPerMonth) * (1 - azureDiscount))

Azure AI Search : Storage Tier Cost Per Month 8586.85
Azure AI Search : Semantic Ranker Cost Per Month 879.75
Azure OpenAI : Input Token Cost Per Month 1543.6799999999998
Azure OpenAI : Output Token Cost Per Month 887.616
Total Azure AI Search Cost 9466.6
Total Azure OpenAI Cost 2431.296
Total Cost Per Month for One Use-Case 11897.896

---------------Multiple Similar Use-Cases--------------------
Total cost for Azure AI Search for 16 UseCases 151465.6
Total cost for Azure OpenAI for 16 UseCases 38900.736
Total cost for 16 UseCases 190366.336

---------------After Discount--------------------
Total cost for Azure AI Search for 16 UseCases 49226.32000000001
Total cost for Azure OpenAI for 16 UseCases 12642.7392
Total cost for 16 UseCases 61869.0592
